In [165]:
import json
import pandas as pd
import os
from scipy.ndimage import uniform_filter1d

In [166]:
final_dir = "C:\\Users\\daddyj\\Documents\\Batteryze\\Tarform\\final_data"

data = {}
data["voltage"] = []
data["temperature"] = []
data["current"] = []
data["soc"] = []
data["charge"] = []
data["self_discharge"] = []
data["resistance"] = []
data["cycles"] = []
data["state_of_health"] = []

temp_soh_data = []

In [167]:
for fname in os.listdir(final_dir):
    df = pd.read_csv(os.path.join(final_dir, fname))
    df = df.iloc[::300,:]
    df = df.dropna()
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    df["timestamp"] = df["timestamp"].apply(lambda x: x.timestamp()*1000)
   
    a = df.apply(lambda x: {"x": x["timestamp"], "y": float("{:.2f}".format(x["voltage"]))}, axis=1)
    data["voltage"] += list(a)

    b = df.apply(lambda x: {"x": x["timestamp"], "y": float("{:.2f}".format(x["temperature"]))}, axis=1)
    data["temperature"] += list(b)

    data["current"] += list(df.apply(lambda x: {"x": x["timestamp"], "y": float("{:.2f}".format(x["current"]))}, axis=1))
    data["soc"] += list(df.apply(lambda x: {"x": x["timestamp"], "y": float("{:.2f}".format(x["soc"]))}, axis=1))
    data["charge"] += list(df.apply(lambda x: {"x": x["timestamp"], "y": float("{:.2f}".format(x["charge"]))}, axis=1))

    # df.loc[df["resistance"] < 0, "resistance"] = 0.0
    # print(df["resistance"].min())
    data["resistance"] += list(df.apply(lambda x: {"x": x["timestamp"], "y": float("{:.2f}".format(max(0, x["resistance"]*1000)))}, axis=1))
    temp_soh_data += list(df["soh"])

In [168]:
temp_soh_data = uniform_filter1d(temp_soh_data, size=5000)

In [169]:
for i in range(len(temp_soh_data)):
    data["state_of_health"].append(
        {
            "x": data["soc"][i]["x"],
            "y": float("{:.2f}".format(temp_soh_data[i]))
        }
    )


In [170]:
json.dump(data, open("C:\\Users\\daddyj\\Documents\\Batteryze\\Batteryze_Frontend\\src\\mock\\data\\Tarform_Test_Battery_2.json", "w+" ))